<a href="https://colab.research.google.com/github/t8101349/Costco-discounted-goods-crawler-bot/blob/main/costco_bot2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 移除重複路徑
!rm -rf /tools/google-cloud-sdk
!rm -rf /root/google-cloud-sdk
!test -d /root/google-cloud-sdk && echo "尚未刪除" || echo "已刪除"


已刪除


In [2]:
# 1. 安裝套件
!pip install flask line-bot-sdk

# 2. 安裝 gcloud CLI（僅需第一次執行）
!curl https://sdk.cloud.google.com | bash
!exec -l $SHELL



串流輸出內容已截斷至最後 5000 行。
google-cloud-sdk/lib/third_party/botocore/data/cloudfront/2015-04-17/endpoint-rule-set-1.json
google-cloud-sdk/lib/third_party/botocore/data/cloudfront/2015-04-17/paginators-1.json
google-cloud-sdk/lib/third_party/botocore/data/cloudfront/2015-04-17/service-2.json
google-cloud-sdk/lib/third_party/botocore/data/cloudfront/2015-04-17/waiters-2.json
google-cloud-sdk/lib/third_party/botocore/data/cloudfront/2015-07-27/endpoint-rule-set-1.json
google-cloud-sdk/lib/third_party/botocore/data/cloudfront/2015-07-27/paginators-1.json
google-cloud-sdk/lib/third_party/botocore/data/cloudfront/2015-07-27/service-2.json
google-cloud-sdk/lib/third_party/botocore/data/cloudfront/2015-07-27/waiters-2.json
google-cloud-sdk/lib/third_party/botocore/data/cloudfront/2015-09-17/endpoint-rule-set-1.json
google-cloud-sdk/lib/third_party/botocore/data/cloudfront/2015-09-17/paginators-1.json
google-cloud-sdk/lib/third_party/botocore/data/cloudfront/2015-09-17/service-2.json
google-cloud-sdk

In [5]:
import os
os.environ['PATH'] += ":/root/google-cloud-sdk/bin"


In [6]:
# 設定環境變量
!export PATH=$PATH:/root/google-cloud-sdk/bin
!ls /root/google-cloud-sdk/bin


bootstrapping	  docker-credential-gcloud  git-credential-gcloud.sh
bq		  gcloud		    gsutil
dev_appserver.py  gcloud-crc32c		    java_dev_appserver.sh


In [7]:
!gcloud init
!gcloud version


Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'true'
compute:
  gce_metadata_read_timeout_sec: '0'
core:
  disable_usage_reporting: 'True'

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

You must sign in to continue. Would you like to sign in (Y/n)?  y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=

In [59]:
# 1. 建立 main.py
with open("main.py", "w") as f:
    f.write('''
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
import os
import requests
import json
import time

app = Flask(__name__)

LINE_CHANNEL_ACCESS_TOKEN = os.getenv('LINE_CHANNEL_ACCESS_TOKEN')
LINE_CHANNEL_SECRET = os.getenv('LINE_CHANNEL_SECRET')

line_bot_api = LineBotApi(LINE_CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(LINE_CHANNEL_SECRET)

# Costco 折扣抓取邏輯
api_url = "https://www.costco.com.tw/rest/v2/taiwan/products/search"
params = {
    "fields": "FULL",
    "query": ":BazaarVoiceRating-desc:bazaarVoiceAverageRatingFacet:4-5:bazaarVoiceAverageRatingFacet:5:bazaarVoiceAverageRatingFacet:3-4",
    "pageSize": 3,
    "sort": "BazaarVoiceRating-desc",
    "category": "Coupon",
    "lang": "zh_TW",
    "curr": "TWD",
    "currentPage": 0
}

def fetch_costco_data():
    headers = {"User-Agent": "Mozilla/5.0", "Accept": "application/json"}
    response = requests.get(api_url, headers=headers, params=params)
    products_info = []

    if response.status_code == 200:
        products = response.json().get('products', [])
        for product in products:  #商品資料
            name = product.get("name")
            price = product.get("price", {'value':"售完"}).get("value")
            couponDiscount = product.get("couponDiscount", {'discountValue':"N/A"}).get("discountValue","N/A")
            currency = product.get("price", {}).get("currencyIso")
            rating = product.get("averageRating", "無評價")
            products_info.append(f"item: {name} {price} {currency}\n")


    return products_info

@app.route('/callback', methods=['POST'])
def callback():
    signature = request.headers.get('X-Line-Signature')
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_message = event.message.text
    if user_message == "查詢折扣":
        costco_data = fetch_costco_data()
        reply = "\n".join(costco_data) if costco_data else "暫無優惠資訊"
    else:
        reply = f"你說了：{user_message}"
    line_bot_api.reply_message(event.reply_token, TextSendMessage(text=reply))

# 如果部署到 GCP Cloud Functions
def main(request):
    return callback()

# 如果本地端執行 (for Local Test)
if __name__ == "__main__":
    port = int(os.environ.get("PORT", 8080))
    app.run(host="0.0.0.0", port=port, debug=True)
''')


In [40]:
import os

# 設定環境變量
os.environ["LINE_CHANNEL_ACCESS_TOKEN"] = 'ik9reNYdHWJhkqdey1cIrtKNF/yMKWyCGTZmkfeZH4FtGSj/hwZmHputQN/95c5dW29MRScjKcLwRFwD67vw8pMON62xtn/lbxm/7X7jVHplqvOtFM9jAGb+azrpqSEULEzdr4OSCJj2aAYm2wH21AdB04t89/1O/w1cDnyilFU='
os.environ["LINE_CHANNEL_SECRET"] = 'a32b70f15b6fe55ec4ae766e1325cd16'

In [60]:
# 生成依賴檔案
with open("requirements.txt", "w") as f:
    f.write("flask==2.3.0\nline-bot-sdk==3.16.3\nrequests==2.32.3\n")



In [11]:
# 設定 GCP 專案
!gcloud config set project costco-line

# 確認 GCP 認證
!gcloud auth login


Updated property [core/project].
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=BrvqmekJz60qYmu391TOmocAciRKpI&prompt=consent&token_usage=remote&access_type=offline&code_challenge=ueb_0UbDtRYAY1n4hGheax0vmsI3qRxkIP3kafdyfSs&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AQSTgQFBovdNd6TVFMIk3-4QK_PWbw_5SGG2ub8Q8HK2G1GNTMWx36HlI6GE3UeV3L3zsA

You are now logged in as [t8101349@gma

In [61]:
%%writefile Dockerfile
# Step 1: 使用 Python 基礎映像檔
FROM python:3.10-slim

# Step 2: 設定工作目錄
WORKDIR /app

# Step 3: 複製專案檔案
COPY . .

# Step 4: 安裝相依套件
RUN pip install --no-cache-dir -r requirements.txt

# Step 5: 環境變量
ENV PORT 8080
EXPOSE 8080

# Step 6: 啟動應用
CMD ["python", "main.py"]


Overwriting Dockerfile


In [62]:
!gcloud run deploy linebot-function \
  --image gcr.io/[YOUR_PROJECT_ID]/linebot-function \
  --platform managed \
  --region asia-east1 \
  --allow-unauthenticated \
  --timeout=900s  # 增加超時時間
